In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
# slurm joblist to intersect each motif with the Peak file
os.makedirs("../result/peak_vs_motif/PublishedOptimalWormPeaks_motif/", exist_ok=True)
files = os.listdir("/home/jg2447/slayman/motifs/FIMO/ce11_InferAndDirect/")
with open("joblist_fimo", "w") as ff:
    for motif in files:
        ff.write("module load BEDTools; bedtools intersect -a ../data/PeakFiles/PublishedOptimalWormPeaks -b /home/jg2447/slayman/motifs/FIMO/ce11_InferAndDirect/{motif} -c > ../result/peak_vs_motif/PublishedOptimalWormPeaks_motif/{motif}\n".format(motif=motif))

In [ ]:
# all chipped TFs
df = pd.read_csv("../data/PeakFiles/PublishedOptimalWormPeaks", sep="\t", header=None)
TFs = df[3].str.split("_", expand=True)[0].unique()
TFs_noP = [ii.replace("(", "").replace(")", "") for ii in TFs]
noP2P = pd.Series(TFs, index=TFs_noP).to_dict()

# all motifs
files = [ii.replace(".bed", "") for ii in os.listdir("../result/peak_vs_motif/PublishedOptimalWormPeaks_motif/")]

# overlap TF list
overlap_TF_noP = list(set.intersection(set(files), set(TFs_noP)))
overlap_TF_P = [noP2P[ii] for ii in overlap_TF_noP]

In [ ]:
# init empty motif list for each peak
df = pd.read_csv("../result/peak_vs_motif/PublishedOptimalWormPeaks_motif/%s.bed" % files[0], sep="\t", header=None)
target = []
for tt in range(df.shape[0]):
    target.append("")

# add the intersected motif for each peak
for f in overlap_TF_noP:
    df = pd.read_csv("../result/peak_vs_motif/PublishedOptimalWormPeaks_motif/%s.bed" % f, sep="\t", header=None)
    valid_id = np.array(range(df.shape[0]))[df[10] != 0]
    for vid in valid_id:
        target[vid] = target[vid] + "," + noP2P[f]
target_final = [ii.lstrip(",") for ii in target]

# save
with open("../result/peak_vs_motif/PublishedOptimalWormPeaks_motif.txt", "w") as ff:
    for ii in target_final:
        ff.write(ii+"\n")